In [3]:
import pandas as pd     # Se carga la Libreria de pandas

- **Carga de Datasets**

In [4]:
# Se carga el dataset games desde el archivo csv
df_games= pd.read_csv('DataSet_tranformados/games.csv',delimiter = ',',encoding = "utf-8")
df_games.head(10)

,genres,game_name,game_id,release_year
0,Action,Lost Summoner Kitty,761140,2018
1,Casual,Lost Summoner Kitty,761140,2018
2,Indie,Lost Summoner Kitty,761140,2018
3,Simulation,Lost Summoner Kitty,761140,2018
4,Strategy,Lost Summoner Kitty,761140,2018
5,Free to Play,Ironbound,643980,2018
6,Indie,Ironbound,643980,2018
7,RPG,Ironbound,643980,2018
8,Strategy,Ironbound,643980,2018
9,Casual,Real Pool 3D - Poolians,670290,2017


In [5]:
df_games.info()     # Muestra la informacion de las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74832 entries, 0 to 74831
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        71550 non-null  object
 1   game_name     74832 non-null  object
 2   game_id       74832 non-null  int64 
 3   release_year  74832 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [6]:
# Se carga el dataset games desde el archivo csv
df_reviews= pd.read_csv('DataSet_tranformados/reviews.csv',delimiter = ',',encoding = "utf-8")
df_reviews.head()

,user_id,year,game_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2


In [7]:
# Se carga el dataset games desde el archivo csv
df_items= pd.read_parquet('DataSet_tranformados/items.parquet')
df_items.head()

,user_id,game_id,game_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [8]:
df_items.info()     # Muestra la informacion de las columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094092 entries, 0 to 5094091
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   game_id           int32 
 2   game_name         object
 3   playtime_forever  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 136.0+ MB


In [75]:
#df= df_games[df_games['genres'].notna()]
#df['genres2']= df['genres'].apply(ast.literal_eval)

In [17]:
def PlayTimeGenre(genero: str):    
    df_filtro_genero= df_games.copy()       # Copia el df de Games
    df_filtro_genero['genres']= df_filtro_genero['genres'].str.lower()      #Convierte la columan Genero a minusculas
    df_filtro_genero= df_filtro_genero[df_filtro_genero['genres'] == genero.lower()]    # Se filtran los games por el genero de entrada en minusculas
    indice_genero= df_filtro_genero.index[0]            # Se toma el indice de la primera fila del df filtrado
    strGenero= df_games.loc[indice_genero]['genres']    # Guarda el Genero como esta en el df_game original para visualizarlo en el resultado
    df_union = pd.merge(df_filtro_genero, df_items, on='game_name', how= 'inner')   # Se unen (join) los juegos con los items
    # Se agrupa por y año
    df_grupo_por_año = df_union.groupby('release_year', as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    year_max_horas= int(df_grupo_por_año.iloc[0,0])     # Se toma el año de la primera fila del dataframe ordenado
    # Guarda la salida de la funcion a una variable
    result= [{'Año de lanzamiento con más horas jugadas para Género {}'.format(strGenero): year_max_horas}]    
    return result

In [18]:
PlayTimeGenre('STRAtegy')

[{'Año de lanzamiento con más horas jugadas para Género Strategy': 2013}]

In [14]:
def UserForGenre(genero: str):
    df_filtro_genero= df_games.copy()       # Copia el df de Games  
    df_filtro_genero['genres']= df_filtro_genero['genres'].str.lower()      #Convierte la columan Genero a minusculas
    # Se agrupa el dataset "df_Items" por Usuario y juegos y se sumo el tiempo de juef
    df_items_por_user = df_items.groupby(by=['user_id','game_name'], as_index=False)['playtime_forever'].sum() 
    # Se filtra el dataset "df_games" por el genero requerido
    df_filtro_genero= df_filtro_genero[df_filtro_genero['genres'] == genero.lower()]
    indice_genero= df_filtro_genero.index[0]            # Se toma el indice de la primera fila del df filtrado
    strGenero= df_games.loc[indice_genero]['genres']    # Guarda el Genero como esta en el df_game original para visualizarlo en el resultado
    # Se hace una union de los juegos fltrados por el genero requerido y el dataset de items ya filtrado
    df_union = pd.merge(df_filtro_genero, df_items_por_user, on='game_name', how= 'inner')
    # se suman las horas jugadas de cada usuario por año
    df_grupo_por_user_y_año = df_union.groupby(by=['user_id','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    # Se suman las horas jugadas de todos los años por usuario
    df_user_max= df_grupo_por_user_y_año.groupby(by=['user_id'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    user_max_horas= df_user_max.iloc[0,0]   # Se toma el user_id con mas horas jugadas 
    df_horas_jugadas= df_grupo_por_user_y_año[df_grupo_por_user_y_año['user_id']==user_max_horas]   #se realiza un filtro por el usuario con mas horas jugadas
    df_horas_jugadas.sort_values(by='playtime_forever', ascending=False)    #se ordena por tiempo de juego
    
    # Se itera sobre el df donde se estan los años y hora jugadas por año y se guardan en una lista
    list_horas_xyear= [{'Año ': year, 'Horas: ': horas} for year, horas in zip(df_horas_jugadas['release_year'], df_horas_jugadas['playtime_forever'])]
    
    # Se guardan los resultado en la variable result
    result= {'Usuario con mas horas jugadas para el Genero {}'.format(strGenero): user_max_horas,
            'Horas Jugadas':list_horas_xyear}
    
    return result
    
    

In [16]:
UserForGenre('ACTion')

{'Usuario con mas horas jugadas para el Genero Action': 'shinomegami',
 'Horas Jugadas': [{'Año ': 2003, 'Horas: ': 532756},
  {'Año ': 2009, 'Horas: ': 209805},
  {'Año ': 2013, 'Horas: ': 148625},
  {'Año ': 2015, 'Horas: ': 102197},
  {'Año ': 2014, 'Horas: ': 92080},
  {'Año ': 2011, 'Horas: ': 89382},
  {'Año ': 2007, 'Horas: ': 68354},
  {'Año ': 2016, 'Horas: ': 42563},
  {'Año ': 2012, 'Horas: ': 18284},
  {'Año ': 2000, 'Horas: ': 10775},
  {'Año ': 2010, 'Horas: ': 9405},
  {'Año ': 1998, 'Horas: ': 3716},
  {'Año ': 2001, 'Horas: ': 2429},
  {'Año ': 2006, 'Horas: ': 2043},
  {'Año ': 1996, 'Horas: ': 1161},
  {'Año ': 2005, 'Horas: ': 343},
  {'Año ': 1999, 'Horas: ': 303},
  {'Año ': 2008, 'Horas: ': 180},
  {'Año ': 1988, 'Horas: ': 136},
  {'Año ': 2017, 'Horas: ': 135},
  {'Año ': 2004, 'Horas: ': 120},
  {'Año ': 1997, 'Horas: ': 99},
  {'Año ': 1995, 'Horas: ': 2},
  {'Año ': 1993, 'Horas: ': 0},
  {'Año ': 1992, 'Horas: ': 0},
  {'Año ': 1991, 'Horas: ': 0},
  {'Año 

In [59]:
# Funcion para usar el mismo codigo para las funcion de juegos remomendados y no recomendados
def userRecomended_or_Not(año: int, tipo: bytes):
    if tipo == 1:   # Se filtran los juegos remomendados
        df_filtred_rec_sent_year= df_reviews[(df_reviews['recommend']== True) & (df_reviews['sentiment_analysis']> 0) & (df_reviews['year'] == año)]
    else:       #Se filtra por los juego no recomendados
        df_filtred_rec_sent_year= df_reviews[(df_reviews['recommend']== False) & (df_reviews['sentiment_analysis']== 0) & (df_reviews['year'] == año)]
    # Se agupan los reviews por juego y se cuentan
    df_grupo_xgames= df_filtred_rec_sent_year.groupby('game_id', as_index=False).size().sort_values(by='size', ascending=False)
    # se crea un df temporal con los valores unicos de juegos para hacer el join con reviews
    df_games_sin_duplicados = df_games.drop_duplicates(subset=['game_id'])  
    # Hace una union(join) de los reviews filtrados con los juegos
    df_union_con_reviews= pd.merge(df_grupo_xgames, df_games_sin_duplicados, on='game_id', how= 'inner')
    # Se toman los 3 primeros ligares de los juegos con mas review segun el filtro aplicado
    result= {'Puesto {}'.format(pos +1): game for pos, game in zip(range(3),df_union_con_reviews['game_name'].iloc[:3])}
    return result

In [31]:
def UsersRecommend(año: int):   
    # Se filtran los juegos remomendados 
    df_filtred_rec_sent_year= df_reviews[(df_reviews['recommend']== True) & (df_reviews['sentiment_analysis']> 0) & (df_reviews['year'] == año)]
    # Se agupan los reviews por juego y se cuentan
    df_grupo_xgames= df_filtred_rec_sent_year.groupby('game_id', as_index=False).size().sort_values(by='size', ascending=False)
    # se crea un df temporal con los valores unicos de juegos para hacer el join con reviews
    df_games_sin_duplicados = df_games.drop_duplicates(subset=['game_id'])  # se crea una df temporal sin id_games duplicados
    # Hace una union(join) de los reviews filtrados con los juegos
    df_union_con_reviews= pd.merge(df_grupo_xgames, df_games_sin_duplicados, on='game_id', how= 'inner')
    # Se toman los 3 primeros ligares de los juegos con mas review segun el filtro aplicado para la salida
    result= {'Puesto {}'.format(pos +1): game for pos, game in zip(range(3),df_union_con_reviews['game_name'].iloc[:3])}
    return result

In [37]:
UsersRecommend(2010)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'Killing Floor',
 'Puesto 3': 'Alien Swarm'}

In [60]:
def UsersNotRecommend(año: int ):
    # Se filtran los juegos No remomendados 
    df_filtred_rec_sent_year= df_reviews[(df_reviews['recommend']== False) & (df_reviews['sentiment_analysis']== 0) & (df_reviews['year'] == año)]
    # Se agupan los reviews por juego y se cuentan
    df_grupo_xgames= df_filtred_rec_sent_year.groupby('game_id', as_index=False).size().sort_values(by='size', ascending=False)
    # se crea un df temporal con los valores unicos de juegos para hacer el join con reviews
    df_games_sin_duplicados = df_games.drop_duplicates(subset=['game_id'])  
    # Hace una union(join) de los reviews filtrados con los juegos
    df_union_con_reviews= pd.merge(df_grupo_xgames, df_games_sin_duplicados, on='game_id', how= 'inner')
    # Se toman los 3 primeros ligares de los juegos con mas review negativos segun el filtro aplicado
    result= {'Puesto {}'.format(pos +1): game for pos, game in zip(range(3),df_union_con_reviews['game_name'].iloc[:3])}
    return result

In [61]:
UsersNotRecommend(2012)

{'Puesto 1': 'Team Fortress 2',
 'Puesto 2': 'PAYDAY™ The Heist',
 'Puesto 3': "The Kings' Crusade"}

In [36]:
def sentiment_analysis(año: int):
    # se crea un df temporal con los valores unicos de juegos para hacer el join con reviews
    df_games_sin_duplicados = df_games.drop_duplicates(subset=['game_id'])
    # Se filtran los juegos por el año de lanzamiento del parametro de entrada
    df_games_sin_duplicados = df_games_sin_duplicados[df_games_sin_duplicados['release_year'] == año]
    # Hace una union(join) de los juegos resutrados del filtro con los reviews
    df_union_games_rev = pd.merge(df_games_sin_duplicados, df_reviews, on='game_id', how= 'inner')
    # Se agrupan por analisis de sentimiento y se cuentan
    df_grupo_xSententiment= df_union_games_rev.groupby('sentiment_analysis', as_index=False).size().sort_values(by='sentiment_analysis', ascending=True)
    # Se formatean los datos para el resultado y se retornan
    result= {'Negative': df_grupo_xSententiment.iloc[0,1], 'Neutral': df_grupo_xSententiment.iloc[1,1], 'Positive': df_grupo_xSententiment.iloc[2,1]}
    return result

In [43]:
sentiment_analysis(2012)

{'Negative': 1249, 'Neutral': 3066, 'Positive': 4740}

#########################################################################################################

- **Funciones para generar un dataset la funciones del main.py**

In [50]:
def Generar_df_funcion1():
    # Se hace una union (join) de juegos con items
    df_union_games_items = pd.merge(df_games, df_items, on='game_name', how= 'inner')
    # se crea un dataframe agrupado por genero y año y se sumas las horas jugadas
    df_grupo_por_genero_y_año = df_union_games_items.groupby(by=['genres','release_year'], as_index=False)['playtime_forever'].sum().sort_values(by='playtime_forever', ascending=False)
    # Se exporta el datagrame a un archivo llamado funcion1.csv desde donde se cargaran el dataset para la funcion 1 de la api
    df_grupo_por_genero_y_año.to_csv('DataSet_tranformados/funcion1.csv', index=False)    
    return

In [49]:
Generar_df_funcion1()

In [ ]:
def Generar_df_funcion2():
    # Se hace una union (join) de juegos con items
    df_union_game_items = pd.merge(df_games, df_items, on='game_id', how= 'inner')
    # se crea un dataframe agrupado por genero, usuario y año y se sumas las horas jugadas
    df_grupo_por_gen_us_year = df_union_game_items.groupby(by=['genres','user_id','release_year'], as_index=False)['playtime_forever'].sum()
    # Se exporta el datagrame a un archivo en formato parquet llamado funcion2.parquet desde donde se cargaran el dataset para la funcion 2 de la api
    df_grupo_por_gen_us_year.to_parquet('DataSet_tranformados/funcion2.parquet', index=False)
    return

In [ ]:
Generar_df_funcion2()